<a href="https://colab.research.google.com/github/aaditkamat/analytics-projects/blob/main/Online_Shoppers_RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [159]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [160]:
# Read data from
data = pd.read_csv('/content/drive/MyDrive/ZeroToMastery/Business Analytics/Segmentation/RFM/online_shoppers.csv', encoding='ISO-8859-1').dropna()
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [161]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


In [162]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [163]:
# Remove rows where  quantity is negative
data = data.loc[data['Quantity'] >=0, :]

In [164]:
# Remove rows where customer id is null
data.dropna(subset='CustomerID', inplace=True)

<ipython-input-164-bfaff4409b77>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(subset='CustomerID', inplace=True)


In [165]:
# Set correct types for the different columns
data['InvoiceDate'] = data['InvoiceDate'].astype('datetime64[ns]')
data.loc[:, 'CustomerID'] = np.int64(data.loc[:, 'CustomerID'])
data

<ipython-input-165-40f6785df0fa>:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.loc[:, 'CustomerID'] = np.int64(data.loc[:, 'CustomerID'])


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


In [166]:
# Create sales column
data['Sales'] = data['UnitPrice'] * data['Quantity']

In [167]:
# Get the last date available to calculate recency
snapshot_date = data['InvoiceDate'].max()
snapshot_date += datetime.timedelta(days = 1)

In [168]:
# Set recency as the number of days since last purchase
grouped_data = data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'Sales': 'sum'
})

In [169]:
# Create a monetary column
grouped_data['Monetary'] = grouped_data['Sales'] / grouped_data['InvoiceNo']
grouped_data.head()

,InvoiceDate,InvoiceNo,Sales,Monetary
CustomerID,,,,
12346,326,1,77183.60,77183.600000
12347,2,182,4310.00,23.681319
12348,75,31,1797.24,57.975484
12349,19,73,1757.55,24.076027
12350,310,17,334.40,19.670588


In [170]:
# Creating RFM columns
grouped_data.drop(columns='Sales', inplace=True)
grouped_data.rename(columns = {
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency'
}, inplace=True)

# RFM Preparation

In [171]:
# Set columns that are supposed to be sorted in ascending order, with labels that are in descending order
# This is because the bins are formed in ascending order and the last rank is given to the first bin
# and first rank to the last bin
# Similarly, set columns that are supposed to be sorted in descending order, with labels in ascneding order
columns = grouped_data.columns
for col in columns[1: ]:
  grouped_data[f'{col}_Rank'] = pd.qcut(grouped_data[f'{col}'], 4, labels=list(range(4, 0, -1)))
grouped_data[f'{columns[0]}_Rank'] = pd.qcut(grouped_data[f'{columns[0]}'], 4, labels=list(range(1, 5)))
grouped_data.head()

,Recency,Frequency,Monetary,Frequency_Rank,Monetary_Rank,Recency_Rank
CustomerID,,,,,,
12346,326,1,77183.600000,4,1,4
12347,2,182,23.681319,1,2,1
12348,75,31,57.975484,3,1,3
12349,19,73,24.076027,2,2,2
12350,310,17,19.670588,4,2,4


In [172]:
# Calculate an RFM Score
grouped_data['RFM_Score'] = grouped_data[[col for col in grouped_data.columns if 'Rank' in col]].sum(axis=1)
grouped_data.head()

,Recency,Frequency,Monetary,Frequency_Rank,Monetary_Rank,Recency_Rank,RFM_Score
CustomerID,,,,,,,
12346,326,1,77183.600000,4,1,4,9
12347,2,182,23.681319,1,2,1,4
12348,75,31,57.975484,3,1,3,7
12349,19,73,24.076027,2,2,2,6
12350,310,17,19.670588,4,2,4,10


In [173]:
# Create RFM Segmentation function based on custom logic
def rfm_segmentation(row):
  if row['RFM_Score'] >= 11:
    return 'Superstar'
  if row['RFM_Score'] >= 8:
    return 'Future Champion'
  if row['RFM_Score'] >= 6:
    return 'High Potential'
  return 'Low Relevance'

In [174]:
grouped_data['RFM_Segment'] = grouped_data.apply(rfm_segmentation, axis=1)
grouped_data.head()

,Recency,Frequency,Monetary,Frequency_Rank,Monetary_Rank,Recency_Rank,RFM_Score,RFM_Segment
CustomerID,,,,,,,,
12346,326,1,77183.600000,4,1,4,9,Future Champion
12347,2,182,23.681319,1,2,1,4,Low Relevance
12348,75,31,57.975484,3,1,3,7,High Potential
12349,19,73,24.076027,2,2,2,6,High Potential
12350,310,17,19.670588,4,2,4,10,Future Champion


In [175]:
grouped_data.groupby('RFM_Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'RFM_Score': 'count'
})

,Recency,Frequency,Monetary,RFM_Score
RFM_Segment,,,,
Future Champion,138.542294,35.989102,84.372039,1927
High Potential,37.075145,130.933526,72.911331,1384
Low Relevance,15.151762,193.100271,40.120502,738
Superstar,248.179310,16.734483,11.263547,290
